In [1]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from functools import partial, wraps

import plasticc.xgb_train as xgb
import plasticc.lgbm_train as lgbm
from plasticc.featurize import process_meta
from plasticc.featurize import featurize
from plasticc.lgbm_train import lgbm_modeling_cross_validation
from plasticc.xgb_train import xgb_modeling_cross_validation
from plasticc.final import process_test

In [2]:
fcp = {
    'flux': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,
        'mean_change': None,
        'mean_abs_change': None,
        'length': None,
    },

    'flux_by_flux_ratio_sq': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,       
    },

    'flux_passband': {
        'fft_coefficient': [
                {'coeff': 0, 'attr': 'abs'}, 
                {'coeff': 1, 'attr': 'abs'}
            ],
        'kurtosis' : None, 
        'skewness' : None,
    },

    'mjd': {
        'maximum': None, 
        'minimum': None,
        'mean_change': None,
        'mean_abs_change': None,
    },
}

In [3]:
aggs = {
    'flux': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'detected': ['mean'],
    'flux_ratio_sq':['sum', 'skew'],
    'flux_by_flux_ratio_sq':['sum','skew'],
}

## Data processing

In [4]:
%%time
meta_train = process_meta('../data/raw/training_set_metadata.csv')
train = pd.read_csv('../data/raw/training_set.csv')

CPU times: user 1.06 s, sys: 65.6 ms, total: 1.12 s
Wall time: 1.12 s


In [5]:
%%time
X = featurize(train, meta_train, aggs, fcp, n_jobs=4)
X_backup = X.copy()

Feature Extraction: 100%|██████████| 20/20 [00:00<00:00, 30.62it/s]


CPU times: user 15.6 s, sys: 704 ms, total: 16.3 s
Wall time: 19.3 s


In [6]:
if 'target' in X:
    y = X['target']
    del X['target']
else:
    print("What the duck")
    3//0

In [7]:
# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
classes = sorted(y.unique())
class_weights = {c: 1 for c in classes}
class_weights.update({c:2 for c in [64, 15]})
print('Unique classes : {}, {}'.format(len(classes), classes))
print(class_weights)

Unique classes : 14, [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
{6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}


In [8]:
%%time
train_mean = X.mean(axis=0)
#train_mean.to_hdf('train_data.hdf5', 'data')
pd.set_option('display.max_rows', 500)
#import pdb; pdb.set_trace()
X.fillna(0, inplace=True)

CPU times: user 7.49 ms, sys: 88 µs, total: 7.58 ms
Wall time: 7.06 ms


## Train model with CV

In [9]:
eval_func = partial(
    lgbm_modeling_cross_validation, 
    X=X, 
    y=y, 
    classes=classes, 
    class_weights=class_weights, 
    nr_fold=5, 
    random_state=1
)

In [10]:
lgbm_params = {
    'device': 'cpu',
    'objective': 'multiclass',
    'num_class': 14,
    'boosting_type': 'gbdt',
    'n_jobs': 16,
    'max_depth': 7,
    'n_estimators': 1024,
    'subsample_freq': 2,
    'subsample_for_bin': 5000,
    'min_data_per_group': 100,
    'max_cat_to_onehot': 4,
    'cat_l2': 1.0,
    'cat_smooth': 59.5,
    'max_cat_threshold': 32,
    'metric_freq': 10,
    'verbosity': -1,
    'metric': 'multi_logloss',
    'xgboost_dart_mode': False,
    'uniform_drop': False,
    'colsample_bytree': 0.5,
    'drop_rate': 0.173,
    'learning_rate': 0.0267,
    'max_drop': 5,
    'min_child_samples': 10,
    'min_child_weight': 100.0,
    'min_split_gain': 0.1,
    'num_leaves': 7,
    'reg_alpha': 0.1,
    'reg_lambda': 0.00023,
    'skip_drop': 0.44,
    'subsample': 0.75
}


xgb_params = {
    'objective': 'multiclass',
    'booster': 'gbdtree',
    'n_jobs': 16,
    'max_depth': 7,
    'n_estimators': 1024,
    'verbosity': -1,
    'colsample_bytree': 0.5,
    'learning_rate': 0.0267,
    'min_child_weight': 100.0,
    'reg_alpha': 0.1,
    'reg_lambda': 0.00023,
    'subsample': 0.75
}

In [11]:
%%time
# modeling from CV
clfs, score = eval_func(lgbm_params)

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.719458	training's wloss: 0.712412	valid_1's multi_logloss: 1.05447	valid_1's wloss: 0.884753
[200]	training's multi_logloss: 0.464389	training's wloss: 0.45543	valid_1's multi_logloss: 0.827313	valid_1's wloss: 0.682208
[300]	training's multi_logloss: 0.363731	training's wloss: 0.354034	valid_1's multi_logloss: 0.747372	valid_1's wloss: 0.625374
[400]	training's multi_logloss: 0.306134	training's wloss: 0.296357	valid_1's multi_logloss: 0.707577	valid_1's wloss: 0.607326
[500]	training's multi_logloss: 0.264831	training's wloss: 0.255135	valid_1's multi_logloss: 0.680097	valid_1's wloss: 0.601133
Early stopping, best iteration is:
[508]	training's multi_logloss: 0.262046	training's wloss: 0.252407	valid_1's multi_logloss: 0.677936	valid_1's wloss: 0.600375
no 1-fold loss: 0.6003746421112547
Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.719866

In [12]:
filename = 'subm_{:.6f}_{}.csv'.format(
    score, 
    dt.now().strftime('%Y-%m-%d-%H-%M')
)
print('save to {}'.format(filename))

save to subm_0.609619_2018-12-09-15-47.csv


## FINAL RUN

In [13]:
# calculate test size
!wc -l ../data/raw/test_set_sample.csv

1000001 ../data/raw/test_set_sample.csv


In [14]:
#chunk_size_one_100 = 453653105 // 100 + 1
chunk_size_one_100 = 1000001 // 100 + 1
chunk_size_one_100

10001

In [19]:
%%time
# should take 100x (time after 1st iteration)
process_test(
    clfs, 
    features=X.columns, 
    featurize_configs={'aggs': aggs, 'fcp': fcp}, 
    train_mean=train_mean, 
    output_path=filename,
    meta_path='../data/raw/test_set_metadata.csv',
    test_path='../data/raw/test_set_sample.csv',
    chunks=chunk_size_one_100,
    n_jobs=4
)

Feature Extraction: 100%|██████████| 15/15 [00:00<00:00, 8467.64it/s]


          10001 done in   0.1 minutes


Feature Extraction: 100%|██████████| 16/16 [00:00<00:00, 54295.20it/s]


          20002 done in   0.1 minutes


Feature Extraction: 100%|██████████| 15/15 [00:00<00:00, 9833.47it/s]


...

         990099 done in   3.0 minutes


Feature Extraction: 100%|██████████| 15/15 [00:00<00:00, 10510.28it/s]


        1000100 done in   3.0 minutes


Feature Extraction: 100%|██████████| 1/1 [00:00<00:00, 1205.26it/s]


CPU times: user 1min 29s, sys: 53.5 s, total: 2min 22s
Wall time: 3min 3s


In [20]:
z = pd.read_csv(filename)
print("Shape BEFORE grouping: {}".format(z.shape))
z = z.groupby('object_id').mean()
print("Shape AFTER grouping: {}".format(z.shape))
z.to_csv('single_{}'.format(filename), index=True)

Shape BEFORE grouping: (3036, 16)
Shape AFTER grouping: (3036, 15)
